In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, hamming_loss
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.multiclass import OneVsRestClassifier
import lightgbm as lgb
from sentence_transformers import SentenceTransformer
# Agent 1: Data Loader Agent
class DataLoaderAgent:
    def __init__(self, path, subset_ratio=1.0, random_state=42):
        self.path = path
        self.subset_ratio = subset_ratio
        self.random_state = random_state

    def execute(self):
        df = pd.read_csv(self.path)
        print(f"[DataLoaderAgent] Full Dataset Loaded: {df.shape}")

        if self.subset_ratio < 1.0:
            df = df.sample(frac=self.subset_ratio, random_state=self.random_state)
            print(f"[DataLoaderAgent] Subset Selected: {df.shape}")

        return df
# Agent 2: Preprocessor Agent
class PreprocessorAgent:
    def execute(self, df):
        df["Code"] = df["Code"].astype(str).apply(lambda x: x[:800])

        code = df["Code"]
        labels = df[["Long method", "God class", "Feature envy", "Data class"]]

        metrics = df.drop(columns=[
            "Code", "File", "Project", "Class",
            "Long method", "God class", "Feature envy", "Data class"
        ], errors="ignore").fillna(0)

        return code, metrics, labels
# Agent 3: Multi-LLM Embedding Agent
class MultiLLMEmbedder:

    def embed_batch(self, model, texts, batch_size=300):
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = list(texts[i:i+batch_size])
            emb = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
            embeddings.append(emb)
        return np.vstack(embeddings)

    def execute(self, X_code):

        print("\n[Embedding] TF-IDF...")
        tfidf = TfidfVectorizer(max_features=400)
        tfidf_emb = tfidf.fit_transform(X_code).toarray()

        print("[Embedding] Hashing...")
        hash_vec = HashingVectorizer(n_features=400)
        hash_emb = hash_vec.transform(X_code).toarray()

        print("[Embedding] MiniLM (Loading)...")
        model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L3-v2")
        cache_path = "minilm_cache.npy"

        if os.path.exists(cache_path):
            print("🔁 Loading cached MiniLM embeddings...")
            minilm_emb = np.load(cache_path)
        else:
            print("⏳ Computing MiniLM embeddings (first run only)...")
            minilm_emb = self.embed_batch(model, X_code)
            np.save(cache_path, minilm_emb)
            print("💾 Saved MiniLM cache")

        return {"tfidf": tfidf_emb, "hashing": hash_emb, "minilm": minilm_emb}
# Agent 4: Feature Fusion Agent
class FeatureFusionAgent:
    def execute(self, emb_dict, metrics):
        scaler = StandardScaler()
        metrics_scaled = scaler.fit_transform(metrics)

        return {
            name: np.hstack((emb, metrics_scaled))
            for name, emb in emb_dict.items()
        }
# Helper: Per-label Threshold Tuning
def tune_thresholds(y_true, y_probs):
    n_labels = y_true.shape[1]
    thresholds = np.zeros(n_labels)

    for i in range(n_labels):
        top_f1 = 0
        for t in np.linspace(0.1, 0.9, 81):
            preds = (y_probs[:, i] >= t).astype(int)
            f = f1_score(y_true[:, i], preds, zero_division=0)
            if f > top_f1:
                top_f1 = f
                thresholds[i] = t
    return thresholds
# Agent 5: Agentic Model Trainer (LightGBM + Threshold Tuning)
class AgenticModelTrainer:
    def execute(self, fused, y):
        results = {}
        best_f1 = -1
        best_model = None
        best_name = None
        best_thresholds = None

        print("\n[Model Selection] LightGBM + Agentic Thresholding...")

        for name, X in fused.items():
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42)

            clf = OneVsRestClassifier(
                lgb.LGBMClassifier(
                    n_estimators=250,
                    class_weight="balanced",
                    n_jobs=-1
                )
            )
            clf.fit(X_train, y_train)

            y_probs = np.column_stack([
                est.predict_proba(X_test)[:, 1]
                for est in clf.estimators_
            ])

            thresholds = tune_thresholds(y_test.values, y_probs)
            y_pred = (y_probs >= thresholds).astype(int)

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average="micro", zero_division=0)
            ham = hamming_loss(y_test, y_pred)

            results[name] = (acc, f1, ham)
            print(f" → {name}: Accuracy={acc:.4f}, F1={f1:.4f}, Hamming={ham:.4f}")

            if f1 > best_f1:
                best_f1 = f1
                best_model = clf
                best_name = name
                best_thresholds = thresholds

        print(f"\n🤖 Agent Selected Best Embedding: {best_name} (F1={best_f1:.4f})")
        return (best_model, best_thresholds), results

# Orchestrator Agent
class AgenticOrchestrator:
    def __init__(self, path, subset_ratio=1.0):
        self.path = path
        self.subset_ratio = subset_ratio

    def run(self):
        df = DataLoaderAgent(self.path, self.subset_ratio).execute()
        code, metrics, labels = PreprocessorAgent().execute(df)
        embeddings = MultiLLMEmbedder().execute(code)
        fused = FeatureFusionAgent().execute(embeddings, metrics)
        return AgenticModelTrainer().execute(fused, labels)

# RUN
dataset_path = r"C:\Users\user\Desktop\combined-multi-smell-dataset.csv"

print("\n🚀 Running Full Agentic Multi-LLM System...\n")
(best_model, best_thresholds), results = AgenticOrchestrator(dataset_path).run()
print("\n✔ Completed! Best Model + Thresholds Saved Internally.")



🚀 Running Full Agentic Multi-LLM System...

[DataLoaderAgent] Full Dataset Loaded: (94006, 22)

[Embedding] TF-IDF...
[Embedding] Hashing...
[Embedding] MiniLM (Loading)...


C:\Users\user\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


⏳ Computing MiniLM embeddings (first run only)...
💾 Saved MiniLM cache

[Model Selection] LightGBM + Agentic Thresholding...
[LightGBM] [Info] Number of positive: 429, number of negative: 74775
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.212425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 87012
[LightGBM] [Info] Number of data points in the train set: 75204, number of used features: 414
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 3418, number of negative: 71786
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.199242 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 87012
[LightGBM] [Info] Number of data points in the train set: 75204, number of used features: 414
[LightGBM] [Info] [bin